In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dataset.csv")
df.rename(columns={'Real': 'Prediction', '40':'SpoofType', '41':'Illumination', '42':'Environment', '43':'Spoof'}, inplace=True)
# the prediction by silent face takes values 0-2. Gotta convert it.
df['Prediction'] = df['Prediction'].replace({0.0: 1, 1.0: 0, 2.0: 1})
df['Prediction'] = df['Prediction'].astype(int)
df.head(10)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,36,37,38,39,SpoofType,Illumination,Environment,Spoof,Prediction,Prediction Score
0,Data/test/6964/spoof/494405.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,2,2,1,0,0.997231
1,Data/test/6407/spoof/494411.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,1,2,1,0,0.604510
2,Data/test/6153/spoof/494412.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,1,1,1,1,0.378704
3,Data/test/6411/live/494418.png,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.997799
4,Data/test/6336/spoof/494419.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,1,1,1,0,0.954367
5,Data/test/5162/live/494421.png,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0.654609
6,Data/test/6074/live/494424.png,0,0,1,1,0,0,1,1,1,...,0,0,1,0,0,0,0,0,0,0.989915
7,Data/test/7483/live/494428.png,0,1,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,0,0,0.989566
8,Data/test/9735/spoof/494436.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,7,1,1,1,1,0.998543
9,Data/test/4943/live/494437.png,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0.999986


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26000 entries, 0 to 25999
Data columns (total 47 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        26000 non-null  object 
 1   0                 26000 non-null  int64  
 2   1                 26000 non-null  int64  
 3   2                 26000 non-null  int64  
 4   3                 26000 non-null  int64  
 5   4                 26000 non-null  int64  
 6   5                 26000 non-null  int64  
 7   6                 26000 non-null  int64  
 8   7                 26000 non-null  int64  
 9   8                 26000 non-null  int64  
 10  9                 26000 non-null  int64  
 11  10                26000 non-null  int64  
 12  11                26000 non-null  int64  
 13  12                26000 non-null  int64  
 14  13                26000 non-null  int64  
 15  14                26000 non-null  int64  
 16  15                26000 non-null  int64 

In [4]:
import torch
from torchsummary import summary
from src.anti_spoof_predict import AntiSpoofPredict

model_path = "resources/anti_spoof_models/2.7_80x80_MiniFASNetV2.pth"
device_id = 0

model = AntiSpoofPredict(device_id)
model._load_model(model_path)

summary(model.model, input_size=(3, 80, 80))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 40, 40]             864
       BatchNorm2d-2           [-1, 32, 40, 40]              64
             PReLU-3           [-1, 32, 40, 40]              32
        Conv_block-4           [-1, 32, 40, 40]               0
            Conv2d-5           [-1, 32, 40, 40]             288
       BatchNorm2d-6           [-1, 32, 40, 40]              64
             PReLU-7           [-1, 32, 40, 40]              32
        Conv_block-8           [-1, 32, 40, 40]               0
            Conv2d-9          [-1, 103, 40, 40]           3,296
      BatchNorm2d-10          [-1, 103, 40, 40]             206
            PReLU-11          [-1, 103, 40, 40]             103
       Conv_block-12          [-1, 103, 40, 40]               0
           Conv2d-13          [-1, 103, 20, 20]             927
      BatchNorm2d-14          [-1, 103,

In [13]:
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self, original_model):
        super(CustomModel, self).__init__()
        # all the layers except the final linear layer:
        self.features = nn.Sequential(*list(original_model.children())[:-1])

    def forward(self, x):
        x = self.features(x)
        return x
    
def copy_weights(original_model, custom_model):
    original_state_dict = original_model.state_dict()
    custom_state_dict = custom_model.state_dict()

    for name, param in original_state_dict.items():
        if name in custom_state_dict:
            custom_state_dict[name].copy_(param)

# custom_model = CustomModel(model.model).to(model.device)


In [43]:
from src.data_io import transform as trans
import cv2
from src.generate_patches import CropImage

image_cropper = CropImage()

def dropout_embeddings(image_path):
    custom_model = CustomModel(model.model).to(model.device)
    copy_weights(model.model, custom_model)

    test_transform = trans.Compose([
        trans.ToTensor(),
    ])

    image = cv2.imread(image_path)
    image_bbox = model.get_bbox(image)
    param = {
        "org_img": image,
        "bbox": image_bbox,
        "scale": 2.7,
        "out_w": 80,
        "out_h": 80,
        "crop": True,
    }
    image = image_cropper.crop(**param)
    image = test_transform(image).unsqueeze(0).to(model.device)

    with torch.no_grad():
        custom_model.eval()
        embeddings = custom_model(image)

    return embeddings

embeddings = dropout_embeddings(image_path="/Users/kassymmukhanbetiyar/Development/Verigram/CelebA/archive/CelebA_Spoof_/CelebA_Spoof/Data/test/3613/spoof/511091.png")
print(embeddings.shape)
print(embeddings)

torch.Size([1, 128])
tensor([[ 0.1310,  0.1660,  0.1006, -0.0819,  0.1667,  0.0488,  0.6602,  0.1173,
         -0.7643, -0.1707, -0.7049, -0.0510, -0.1475,  0.0721, -0.1950,  0.1670,
          0.0923, -0.2653, -0.6221, -0.1060,  0.0711, -0.1355, -0.1700,  0.1159,
          0.7886, -0.1630, -0.1112, -0.1767,  0.0223, -0.5466,  0.1688,  0.0645,
         -0.8148, -0.2539, -0.6729, -0.1108, -0.1904, -0.8366,  0.1519, -0.6534,
          0.1240, -0.3364,  0.6816, -0.1915, -0.1469,  0.1535, -0.1138, -0.6000,
         -0.7964, -0.2467, -0.1123,  0.1097,  0.6786,  0.1730,  0.1163,  0.2005,
         -0.0345, -0.0315, -0.1819, -0.1388, -0.7876, -0.1162, -0.1113,  0.6979,
         -0.1944,  0.1723,  0.7779, -0.6513,  0.1556,  0.0240, -0.1755, -0.7101,
         -0.6747,  0.6560, -0.0467, -0.1367, -0.7631,  0.0932, -0.6988, -0.7671,
          0.2518,  0.7703,  0.7701, -0.6726, -0.1162,  0.7533,  0.7100,  0.2321,
          0.7611, -0.1668,  0.7993,  0.0998,  0.0824, -0.0888, -0.2074, -0.1750,
       

In [42]:
test_label = pd.read_json('test_label.json')
print(test_label.shape)

(44, 25758)


In [50]:
def fill_dropout_csv(model_path="resources/anti_spoof_models/2.7_80x80_MiniFASNetV2.pth", checkpoint_path="dropout_embeddings.csv"):
    # Get the number of processed images
    processed_count = 0
    if os.path.isfile(checkpoint_path):
        processed_count = sum(1 for _ in open(checkpoint_path)) - 1  # Subtract 1 to exclude the header row

    # Get the remaining image paths to process
    with open('test_label.json', 'r') as file:
        test_labels = json.load(file)

    remaining_image_names = list(test_labels.keys())[processed_count:]

    # Create a new DataFrame for the newly processed images
    new_embeddings_df = pd.DataFrame(columns=range(1, 129))

    total_iterations = len(remaining_image_names)
    progress_bar = tqdm(total=total_iterations, ncols=80, dynamic_ncols=True, smoothing=0.5)
    # Initialize start time
    start_time = time.time()

    # Iterate over the remaining image paths and fill in the embeddings
    for i, image_name in enumerate(remaining_image_names, start=processed_count + 1):
        image_path = image_dir + image_name
        embeddings = dropout_embeddings(image_path)

        # Add the embeddings to the new DataFrame
        new_embeddings_df.loc[image_name] = embeddings.reshape(1, -1).tolist()[0]  # Reshape to (1, 128)

        # Append the new embeddings to the existing DataFrame and CSV file every 250 iterations
        if i % 250 == 0 or i == len(remaining_image_names):
            new_embeddings_df.to_csv(checkpoint_path, mode='a', header=(i == 250))
            new_embeddings_df = pd.DataFrame(columns=range(1, 129))
            # Update progress bar
            progress_bar.update(250 if i % 250 == 0 else i % 250)
            progress_bar.set_postfix({'Remaining': f'{total_iterations - i}', 'Elapsed': f'{time.time() - start_time:.2f}s'})
    return checkpoint_path

fill_dropout_csv()

KeyboardInterrupt: 

In [59]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt

# Load embeddings from CSV file
df = pd.read_csv('dropout_embeddings.csv', index_col=0)
df = df[:100]
X = df.to_numpy()


# Compute pairwise cosine distances
dist_matrix = 1 - cosine_similarity(X)
# dist_matrix = np.triu(dist_matrix) # only keep upper triangle

# Initialize clustering algorithm
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.1, linkage='complete')

# Fit algorithm to distance matrix
clustering.fit(dist_matrix)

# Extract flat clusters
threshold = 0.5 # set your desired threshold here
labels = clustering.fcluster(clustering.children_, t=threshold, criterion='distance')

# Visualize clusters
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=labels)
plt.show()

AttributeError: 'AgglomerativeClustering' object has no attribute 'fcluster'